# Hello, FABRIC: Create Your First FABRIC Experiment

This Jupyter notebook will walk you through creating your first FABRIC experiment. The "Hello, FABRIC" experiment tests the configuration of your environment to ensure you can create and access resources on FABRIC.  Specifically, the experiment deploys a slice of resources with a single virtual machine and confirms you can log into the virtual machine using FABLib methods.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](./configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
#fablib.show_config(output='jupyter', fields=['bastion_username','slice_private_key_file','bastion_public_addr'])
fablib.show_config()



credmgr_host,cm.fabric-testbed.net
orchestrator_host,orchestrator.fabric-testbed.net
fabric_token,/home/fabric/.tokens.json
project_id,990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username,pruth_0000000529
bastion_key_filename,/home/fabric/work/fabric_config/fabric_bastion_key
bastion_public_addr,bastion-1.fabric-testbed.net
bastion_passphrase,None
slice_public_key_file,/home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file,/home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase,None


credmgr_host,cm.fabric-testbed.net
orchestrator_host,orchestrator.fabric-testbed.net
fabric_token,/home/fabric/.tokens.json
project_id,990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username,pruth_0000000529
bastion_key_filename,/home/fabric/work/fabric_config/fabric_bastion_key
bastion_public_addr,bastion-1.fabric-testbed.net
bastion_passphrase,None
slice_public_key_file,/home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file,/home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase,None


## Step 3 (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.


<img src="./figs/SingleNode.png" width="20%"><br>



In [2]:
try:
    # Create a slice
    slice = fablib.new_slice(name="MySlice3000")

    # Add a node
    for i in range(10):
        slice.add_node(name=f"rocky_node{i}", avoid=['GPN','FIU','CLEM','UCSD','STAR','MAX'], image='default_rocky_8')
    
    for i in range(10):
        slice.add_node(name=f"ubuntu_node{i}", avoid=['GPN','FIU','CLEM','UCSD','STAR','MAX', 'NCSA'], image='default_ubuntu_20')

    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    raise e
    


Retry: 6, Time: 163 sec


ID,d31f21e7-f129-48d7-a26c-cdb461721aab
Name,MySlice3000
Lease Expiration (UTC),2022-10-24 15:56:00 +0000
Lease Start (UTC),2022-10-23 15:56:01 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Site,Host,Cores,RAM,Disk,Image,Management IP,State,Error
f1af8f72-200e-42d1-ab9c-8357df840d8b,rocky_node0,MASS,mass-w3.fabric-testbed.net,2,8,10,default_rocky_8,205.172.170.93,Active,
34ebae76-3dc8-4a9e-8b11-a71edf17f440,rocky_node1,UTAH,utah-w2.fabric-testbed.net,2,8,10,default_rocky_8,2001:1948:417:7:f816:3eff:fec0:2111,Active,
00329214-abe1-48e9-af44-67dd90903c2e,rocky_node2,TACC,tacc-w1.fabric-testbed.net,2,8,10,default_rocky_8,129.114.110.82,Active,
92ec3e80-fca2-43de-a602-53d363397bf8,rocky_node3,MASS,mass-w3.fabric-testbed.net,2,8,10,default_rocky_8,205.172.170.85,Active,
09bd3bab-c510-4c08-9aa8-f4e0a91c9aef,rocky_node4,UTAH,utah-w2.fabric-testbed.net,2,8,10,default_rocky_8,2001:1948:417:7:f816:3eff:fe86:6824,Active,
24722dfe-d8cd-4592-970a-a512ad6fa63d,rocky_node5,SALT,salt-w1.fabric-testbed.net,2,8,10,default_rocky_8,2001:400:a100:3010:f816:3eff:fe60:f2cf,Active,
e107a46f-359c-410e-a48d-63563b3b3f05,rocky_node6,WASH,wash-w1.fabric-testbed.net,2,8,10,default_rocky_8,2001:400:a100:3020:f816:3eff:fe83:1558,Active,
34101898-caf0-4277-b26f-2022541f3689,rocky_node7,SALT,salt-w1.fabric-testbed.net,2,8,10,default_rocky_8,2001:400:a100:3010:f816:3eff:fe65:1127,Active,
390b4970-9f23-4791-a8d7-07871388ba14,rocky_node8,MICH,mich-w3.fabric-testbed.net,2,8,10,default_rocky_8,2607:f018:110:11:f816:3eff:feba:b807,Active,
eac4411d-2369-45a5-a7e8-d8b8778d55ab,rocky_node9,DALL,dall-w3.fabric-testbed.net,2,8,10,default_rocky_8,2001:400:a100:3000:f816:3eff:fe80:ac09,Active,



Time to stable 163 seconds
Running post_boot_config ... Time to post boot config 175 seconds


## Step 5: Observe the Slice's Attributes



In [3]:
file = open("ansible_hosts", "w") 
file.write("[nodes] \n")
for node in slice.get_nodes():
    file.write(f"{node.get_name()} ansible_host={node.get_management_ip()} ansible_user={node.get_username()} ansible_port=22 ansible_ssh_private_key_file={node.get_private_key_file()} \n")
file.close()
    

In [10]:
!time ansible-playbook -f 10 -i ansible_hosts playbook.yaml 


PLAY [Intro to Ansible Playbooks] **********************************************

TASK [Gathering Facts] *********************************************************
ok: [rocky_node6]
ok: [rocky_node3]
ok: [rocky_node0]
ok: [rocky_node8]
ok: [rocky_node2]
ok: [rocky_node9]
ok: [rocky_node1]
ok: [rocky_node4]
ok: [rocky_node7]
ok: [rocky_node5]
ok: [ubuntu_node2]
ok: [ubuntu_node0]
ok: [ubuntu_node6]
ok: [ubuntu_node3]
ok: [ubuntu_node5]
ok: [ubuntu_node7]
ok: [ubuntu_node8]
ok: [ubuntu_node1]
ok: [ubuntu_node9]
ok: [ubuntu_node4]

TASK [Copy file] ***************************************************************
ok: [rocky_node0]
ok: [rocky_node3]
ok: [rocky_node2]
ok: [rocky_node6]
ok: [rocky_node1]
ok: [rocky_node4]
ok: [rocky_node8]
ok: [rocky_node9]
ok: [rocky_node5]
ok: [rocky_node7]
ok: [ubuntu_node0]
ok: [ubuntu_node2]
ok: [ubuntu_node3]
ok: [ubuntu_node6]
ok: [ubuntu_node5]
ok: [ubuntu_node1]
ok: [ubuntu_node4]
ok: [ubuntu_node8]
ok: [ubuntu_node7]
ok: [ubuntu_node9]

TASK [Copy di

In [ ]:



slice = fablib.get_slice(name="MySlice1")
slice.list_nodes(output='jupyter')

### Print the node

Each node in the slice has a set of get functions that return the node's attributes.

In [ ]:
try:
    for node in slice.get_nodes():
        #print(f"{node}")
        node.show()
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
try:
    for node in slice.get_nodes():
        node.execute('echo Hello, FABRIC from node `hostname -s`', output='live')
        #print(stdout)
        
        node.execute('sudo yum install -y net-tools pciutils', output='live')
        

except Exception as e:
    print(f"Exception: {e}")

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")